In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np

import osu.dataset as dataset


osu! path: C:\Users\niooi\AppData\Roaming/../Local/osu!


In [2]:
# reload from saved

xs = np.load(f'.datasets/xs_7500_07-30_22-38-46.npy')
ys = np.load(f'.datasets/ys_7500_07-30_22-38-46.npy')

# xs = np.load(f'.datasets/xs_142_07-21_21-50-55.npy')
# ys = np.load(f'.datasets/ys_142_07-21_21-50-55.npy')

In [3]:
xs.shape

(26648, 2048, 9)

In [3]:
import torch

BATCH_SIZE = 64

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [24]:
import torch
import gc

# Clear GPU cache
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

11

In [23]:
from models.rnn import OsuReplayRNN

rnn = OsuReplayRNN(batch_size=BATCH_SIZE, noise_std=0.045)

# vae = OsuReplayVAE.load(".trained/vae_353rcl_9ft.pt")

rnn.load_data(xs, ys)

Position model compiled with torch.compile
RNN Model initialized on cuda (noise_std=0.045)
Position model parameters: 228162
Data loaded: 21318 training samples, 5330 test samples (position only)


In [25]:
from models.vae import OsuReplayVAE
from models.annealer import Annealer

annealer = Annealer(
    total_steps=10,
    range=(0, 0.38),
    cyclical=True,
    stay_max_steps=5
)

vae = OsuReplayVAE(batch_size=BATCH_SIZE, annealer=annealer, noise_std=0.045)

# vae = OsuReplayVAE.load(".trained/vae_353rcl_9ft.pt")
# vae = OsuReplayVAE.load(".trained/vae_most_recent.pt")

vae.load_data(xs, ys)

VAE models compiled..
VAE Models initialized on cuda (noise_std=0.045)
Encoder parameters: 650016
Decoder parameters: 459122
Data loaded: 21318 training samples, 5330 test samples (position only)


In [32]:
from models.keys import OsuKeyModel

keys = OsuKeyModel(batch_size=BATCH_SIZE)
keys.load_data(xs, ys)

Key Model initialized on cuda (noise_std=0.0)
Key model parameters: 54626
Data loaded: 21318 training samples, 5330 test samples (keys only)


In [33]:
model = keys

EPOCHS = 3

for i in range(25):
    model.train(EPOCHS)
    model.save()
    
model.plot_losses()

Epoch 1/3 (Keys): 100%|██████████| 334/334 [00:42<00:00,  7.91it/s]


Epoch 1/3, Train: 0.4766, Test: 0.4227


Epoch 2/3 (Keys): 100%|██████████| 334/334 [00:38<00:00,  8.64it/s]


Epoch 2/3, Train: 0.4279, Test: 0.4280


Epoch 3/3 (Keys): 100%|██████████| 334/334 [00:38<00:00,  8.59it/s]


...

Epoch 1/3, Train: 0.4089, Test: 0.4096


Epoch 2/3 (Keys): 100%|██████████| 334/334 [00:40<00:00,  8.27it/s]


Epoch 2/3, Train: 0.4071, Test: 0.4077


Epoch 3/3 (Keys):  18%|█▊        | 59/334 [00:07<00:33,  8.15it/s]


KeyboardInterrupt: 

In [ ]:
for i in range(150):
    model.train(EPOCHS)
    model.save()
    
model.plot_losses()

In [28]:
del vae

In [29]:
annealer = Annealer(
    total_steps=10,
    range=(0, 0.25),
    cyclical=True,
    stay_max_steps=5
)

vae = OsuReplayVAE(batch_size=BATCH_SIZE, annealer=annealer, noise_std=0.045)

# vae = OsuReplayVAE.load(".trained/vae_353rcl_9ft.pt")
# vae = OsuReplayVAE.load(".trained/vae_most_recent.pt")

vae.load_data(xs, ys)

model = vae

VAE models compiled..
VAE Models initialized on cuda (noise_std=0.045)
Encoder parameters: 650016
Decoder parameters: 459122
Data loaded: 21318 training samples, 5330 test samples (position only)


In [ ]:
for i in range(175):
    model.train(EPOCHS)
    model.save()
    
model.plot_losses()

In [ ]:
for i in range(300):
    model.train(EPOCHS)
    model.save()
    
model.plot_losses()

In [18]:
# test training results
from osu.rulesets.mods import Mods
import osu.rulesets.beatmap as bm
import osu.dataset as dataset

test_name = 'zuttosummer'
test_mods = 0
test_map_path = f'assets/{test_name}_map.osu'
test_song = f'assets/{test_name}_song.mp3'

test_map = bm.load(test_map_path)
test_map.apply_mods(test_mods)

data = dataset.input_data(test_map)
data = np.reshape(data.values, (-1, dataset.BATCH_LENGTH, len(dataset.INPUT_FEATURES)))
data = torch.FloatTensor(data)

Turning beatmaps into time series data: 100%|██████████| 1/1 [00:00<00:00,  2.56it/s]


In [19]:
replay_data = vae.generate(data)
    
replay_data = np.concatenate(replay_data)
replay_data = np.pad(replay_data, ((0, 0), (0, 2)), mode='constant', constant_values=0)
if not os.path.exists('.generated'):
    os.makedirs('.generated')
    
print(f"Generated replay data shape: {replay_data.shape}")


Generated replay data shape: (12288, 4)


In [20]:
import osu.preview.preview as preview

preview.preview_replay_raw(replay_data, test_map_path, test_mods, test_song)